# Example of using Llamaindex framework for Retrieval Augmented Generation
This notebook shows how to run Llamaindex framework locally to create virtual AI assistant based on RAG (Retrieval Augmented Generation).
For dataset to search for source information wikipedia articles about cryptocurrencies were used.

## Llamaindex setup
### Download llamafile
Download the llamafile with model. Llamafile can contain any LLM. Framework enabled to run it as a local server and use via API. 
TinyLlama-1.1B-Chat-v1.0 model is used for purpose of this example

`wget https://huggingface.co/Mozilla/TinyLlama-1.1B-Chat-v1.0-llamafile/resolve/main/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile?download=true`

Make executable 

`chmod +x TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile`

Run in server mode

`./TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile --server --nobrowser --embedding --port 8081`

### Install Llamaindex Python library

In [ ]:
# Install llama-index
!pip install llama-index 
# Install llamafile integrations and SimpleWebPageReader
!pip install llama-index-embeddings-llamafile llama-index-llms-llamafile llama-index-readers-web

## Configuration

In [ ]:
# Configure LlamaIndex
from llama_index.core import Settings
from llama_index.embeddings.llamafile import LlamafileEmbedding
from llama_index.llms.llamafile import Llamafile
from llama_index.core.node_parser import SentenceSplitter

#configure object to encode text into vector using started endpoint
Settings.embed_model = LlamafileEmbedding(base_url="http://localhost:8081")

#configure object that will use model endpoint
Settings.llm = Llamafile(
    base_url="http://localhost:8081",
    temperature=0,
    seed=0
)

#configure split text to chunks 
Settings.transformations = [
    SentenceSplitter(
        chunk_size=256, 
        chunk_overlap=5
    )
]

In [ ]:
# Load local data with some cryptocurrencies descriptions
from llama_index.core import SimpleDirectoryReader
# create reader for local documents
local_doc_reader = SimpleDirectoryReader(input_dir='./data/cryptocurrency_wikipedia')
# create collection that will contain all documents used for retrieval
docs = local_doc_reader.load_data(show_progress=True)

In [ ]:
# Add Wikipedia pages
from llama_index.readers.web import SimpleWebPageReader
urls = [
    'https://en.wikipedia.org/wiki/Bitcoin',
    'https://en.wikipedia.org/wiki/Ethereum',
    'https://en.wikipedia.org/wiki/Dogecoin'
]
# create reader that can fetch websites content
web_reader = SimpleWebPageReader(html_to_text=True)
# add fetched content to docs collection
docs.extend(web_reader.load_data(urls))

In [ ]:
from llama_index.core import VectorStoreIndex

# create index storage with embedded documents
index = VectorStoreIndex.from_documents(
    docs,
    show_progress=True,
)

# dump storage locally
index.storage_context.persist(persist_dir="./storage")

In [ ]:
# create query type engine to ask questions to llm providing documents from index
query_engine = index.as_query_engine()
# ask questions about data from storage
print(query_engine.query("What is Coinye?"))

In [ ]:
print(query_engine.query("Is Dogecoin stable?"))

In [ ]:
print(query_engine.query("Is Bitcoin good for environment?"))